레이블의 갯수가 두개 이상인 데이터의 분류를 해보자. 그런데 한글 데이터셋을 못구해서... 일단 영어 데이터로 진행하겠다. 이 경우는 6개의 클래스가 있는 `TREC` 데이터를 분석할 것이며, 따라서 `LABEL` 필드에서 `dtype`을 선언할 필요가 없다. 

In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.determinisitic = True

In [2]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

In [3]:
train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)

In [4]:
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [5]:
len(train_data), len(valid_data), len(test_data)

(3816, 1636, 500)

데이터를 살펴보자.

In [6]:
vars(train_data[-3])

{'text': ['Who',
  'is',
  'section',
  'manager',
  'for',
  'guidance',
  'and',
  'control',
  'systems',
  'at',
  'JPL',
  '?'],
 'label': 'HUM'}

In [7]:
vars(train_data[-2])

{'text': ['What',
  'year',
  'did',
  'Jack',
  'Nicklaus',
  'join',
  'the',
  'Professional',
  'Golfers',
  'Association',
  'tour',
  '?'],
 'label': 'NUM'}

단어장을 만들자. 훈련 데이터 갯수가 많지 않아서 최대 갯수 설정은 할 필요가 없긴 하나 그냥 하겠다.

In [13]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                max_size = MAX_VOCAB_SIZE,
                vectors = 'glove.6B.100d',
                unk_init = torch.Tensor.normal_
                )
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [17:28, 822kB/s]                                
100%|█████████▉| 399754/400000 [00:30<00:00, 39243.72it/s]

데이터의 라벨은 다음과 같은 6개이다.

* `HUM` for questions about humans
* `ENTY` for questions about entities
* `DESC` for questions asking you for a description
* `NUM` for questions where the answer is numerical
* `LOC` for questions where the answer is a location
* `ABBR` for questions asking about abbreviations

In [14]:
print(LABEL.vocab.stoi)

defaultdict(None, {'HUM': 0, 'ENTY': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})


이터레이터 설정하자

In [15]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [18]:
next(iter(train_iterator)).text.shape

torch.Size([19, 64])

CNN 모델을 사용해보자.

In [27]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1,
                                             out_channels=n_filters,
                                             kernel_size = (fs, embedding_dim))
                                    for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [sent_len, batch_size]
        text = text.permute(1,0)
        #text = [batch_size, sent_len]
        
        embedded = self.embedding(text)
        #embedded = [batch_size, sent_len, emb_dim]
        
        embedded = embedded.unsqueeze(1)
        #embedded = [batch_size, 1, sent_len, emb_dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        #conv_n = [batch_size, n_filters, sent_len - fs[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch_size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))
        #cat = [batch_size, n_filters * len(filter_size)]
        
        return self.fc(cat)
        

하이퍼파라미터 설정하자. 데이터셋 사이즈가 작으므로 필터 사이즈를 작게 하자.

In [28]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [29]:
model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

파라미터 갯수를 세보자.

In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'이 모델은 {count_parameters(model):,}개의 훈련 가능한 파라미터가 있습니다.')

이 모델은 842,406개의 훈련 가능한 파라미터가 있습니다.


사전훈련된 단어 벡터를 적용하자.

In [32]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [ 0.1638,  0.6046,  1.0789,  ..., -0.3140,  0.1844,  0.3624],
        ...,
        [-0.3110, -0.3398,  1.0308,  ...,  0.5317,  0.2836, -0.0640],
        [ 0.0091,  0.2810,  0.7356,  ..., -0.7508,  0.8967, -0.7631],
        [ 0.4306,  1.2011,  0.0873,  ...,  0.8817,  0.3722,  0.3458]])

unk_token과 pad_token의 벡터를 0으로 만들자.

In [33]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

이 모델은 기존처럼 하나의 확률값을 출력으로 하는 것이 아니라 각 클래스별 확률을 출력하므로, 손실 함수를 기존 `BCEWithLogitsLoss`에서 `CrossEntropyLoss`로 수정해야한다.

In [34]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

각 클래스별 스코어를 출력으로 받으면 가장 높은 점수를 가지는 클래스를 반환하도록 하는 정확도 함수를 만들자. 다만 배치로 작업하고 있으므로, 배치의 평균으로 구하자.

In [35]:
def categorical_accuracy(preds, y):
    """배치별 정확도를 출력하는 함수"""
    max_preds = preds.argmax(dim=1, keepdim=True)
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])    

In [36]:
pred = torch.tensor([[5.0,0.1,0.3],[2.3,3.1,0.7]])
pred

tensor([[5.0000, 0.1000, 0.3000],
        [2.3000, 3.1000, 0.7000]])

In [40]:
pred.argmax(dim=1).shape, pred.argmax(dim=1, keepdim=True).shape

(torch.Size([2]), torch.Size([2, 1]))

In [42]:
pred.argmax(dim=1, keepdim=True).squeeze(1).eq(torch.tensor([1,1]))

tensor([False,  True])

훈련 함수를 만들자.

In [43]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text)
        loss = criterion(predictions, batch.label)
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

eval함수도 비슷

In [46]:
def eval(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)
            loss = criterion(predictions, batch.label)
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [45]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

훈련시키자!

In [48]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = eval(model, train_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tValid Loss: {valid_loss:.3f} | Valid Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 0.292 | Train Acc: 91.53%
	Valid Loss: 0.163 | Valid Acc: 96.73%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.219 | Train Acc: 93.82%
	Valid Loss: 0.117 | Valid Acc: 98.16%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.167 | Train Acc: 95.46%
	Valid Loss: 0.084 | Valid Acc: 98.96%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 0.134 | Train Acc: 96.47%
	Valid Loss: 0.065 | Valid Acc: 98.85%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.104 | Train Acc: 97.44%
	Valid Loss: 0.046 | Valid Acc: 99.48%


테스트 셋에 검증해보자.

In [50]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = eval(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.311 | Test Acc: 88.81%


임의의 문장에 대해 판정해보자.

In [51]:
import spacy
nlp = spacy.load('en')

def predict_class(model, sentence, min_len=4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim=1)
    return max_preds.item()    

실제로 확인해보자.

In [52]:
q = 'The one who told me the truth is Inhwan Lee.'
pred_class = predict_class(model, q)
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 0 = HUM


In [53]:
q = 'The central part of Korea is called Gyung-gi do.'
pred_class = predict_class(model, q)
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 4 = LOC


굿굿!